In [ ]:
# Use this if the training data is compressed as Zip archive
!unzip data.zip

In [ ]:
# Only needed if Google Colab is used
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Be sure to install these in this order or stuff might break
# If the first install dosen't complete relativly fast, use Google Colab 
!pip install tflite-model-maker
!pip install pycocotools

In [4]:
import os
import numpy as np
import tensorflow as tf

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

In [5]:
train_dir = os.path.join('data', 'training')
train_image_dir = os.path.join(train_dir, 'images')
train_annotation_dir = os.path.join(train_dir, 'annotations')

val_dir = os.path.join('data', 'validation')
val_image_dir = os.path.join(val_dir, 'images')
val_annotation_dir = os.path.join(val_dir, 'annotations')

test_dir = os.path.join('data', 'testing')
test_image_dir = os.path.join(test_dir, 'images')
test_annotation_dir = os.path.join(test_dir, 'annotations')

In [6]:
# This is the most precise model (https://www.tensorflow.org/lite/models/modify/model_maker/object_detection#quickstart)
spec = model_spec.get('efficientdet_lite4')

In [7]:
# The label map need to be updated to the correct lables
# Format {x: 'label'}, where x is just ascending numbers
# and label is the label set in the training data

label_map = {1: 'thumbsup'}

train_data = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map=label_map)
val_data = object_detector.DataLoader.from_pascal_voc(val_image_dir, val_annotation_dir, label_map=label_map)
test_data = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_annotation_dir, label_map=label_map)

In [ ]:
# Modify these params for better accuracy
# More epochs take longer, but increases accuracy
# Smaller batch size increases accuracy but can realllllly make it take long mhkay
model = object_detector.create(train_data, model_spec=spec, epochs=40, batch_size=4, train_whole_model=False, validation_data=val_data)

In [ ]:
# Optional
model.evaluate(test_data)

In [ ]:
model.export(export_dir='.') # Results in model.tflite